In [1]:
file_path = 'C:/Users/이다혜/Desktop/2020-2/AI/SMSSpamCollection'

# 파일 읽기
with open(file_path, 'r', encoding='utf8') as inFile:
    lines = inFile.readlines()
    
x_data, y_data = [], []
for line in lines:
    data = line.strip().split('\t')
    label, sentence = data[0], data[1]
    x_data.append(sentence)
    y_data.append(label)

## 1. spam 데이터와 ham 데이터 분류

In [2]:
spam_data, ham_data = [], []
for i in range(len(x_data)):
    if y_data[i] == 'spam':
        spam_data.append(x_data[i])
    else:
        ham_data.append(x_data[i])

## 2. 단어별 카이제곱 통계량 구하기

In [3]:
import re

In [4]:
# 카이제곱 분포를 바탕으로 단어의 가중치를 계산하는 함수
def get_weight(word):
    A, B = 0, 0
    
    # spam 데이터 중에서 word를 포함하는 문서 수 = A
    for i in range(len(spam_data)):
        words = spam_data[i].split()
        if word in words:
            A += 1
            
    # ham 데이터 중에서 word를 포함하는 문서 수 = B
    for i in range(len(ham_data)):
        words = ham_data[i].split()
        if word in words:
            B += 1
    
    C = A - len(spam_data)
    D = B - len(ham_data)
    
    # 카이제곱 통계량 계산하기 
    weight = ((A+B+C+D) * ((A*D-B*C) * (A*D-B*C))) / ((A+B) * (A+C) * (B+D) * (C+D))
    return weight

## 3. 가중치가 높은 200개 단어 선정

In [5]:
chi_squared_weights = {}

for i in range(len(x_data)):
    # 숫자 및 특수문자 제거 등 전처리 
    # sentence = re.sub('[^ a-zA-Z]', ' ', x_data[i])
    # 띄어쓰기가 중복된 경우 하나로 수정, 앞 뒤 공백 제거
    # sentence = re.sub(' +', ' ', sentence).strip()
    
    # 단어의 가중치 딕셔너리 구성 
    # key = word, value = chi squared statistic
    words = x_data[i].split()
    for word in words:
        try:
            chi_squared_weights[word] += get_weight(word)
        except:
            chi_squared_weights[word] = get_weight(word)
        

In [6]:
chi_squared_weights = sorted(chi_squared_weights.items(), key=lambda x : x[1], reverse=True)

In [7]:
# 가중치 상위 200개 단어를 자질로 선정 
features = chi_squared_weights[:200]

In [8]:
features

[('a', 35120.091697687814),
 ('or', 23446.003459718475),
 ('I', 19264.25099821493),
 ('2', 17306.315491896505),
 ('Call', 13770.929692033007),
 ('call', 11631.058365460325),
 ('your', 6916.8863661701935),
 ('ur', 6774.43132232243),
 ('txt', 6736.880268874422),
 ('i', 5722.268531703756),
 ('&', 5553.207311679698),
 ('from', 5444.449140840444),
 ('my', 4543.367118643566),
 ('on', 4172.754223464156),
 ('for', 4020.047460417731),
 ('our', 3977.990553256442),
 ('FREE', 3908.6598770268024),
 ('have', 3487.90753036557),
 ('with', 3289.2918331483093),
 ('4', 2926.2642611610427),
 ('claim', 2587.196275230129),
 ('text', 2458.8876230140786),
 ('To', 2375.9032117551724),
 ('reply', 2334.4538835959816),
 ('me', 2160.289100049933),
 ('now', 2094.246430534518),
 ('mobile', 2055.3091783508803),
 ('Txt', 1952.5437194333365),
 ('contact', 1878.1826525071076),
 ('u', 1840.1282076139153),
 ('Your', 1619.6772803160723),
 ('You', 1513.5616727124216),
 ('you', 1364.6392625353683),
 ('and', 1312.771609667880

## 4. Tokenizer와 위에서 구한 feature를 이용해 문장을 vectorize

In [9]:
from keras.preprocessing.text import Tokenizer

label2index = {'spam':0, 'ham':1}
index2label = {0:'spam', 1:'ham'}

indexing_x_data, indexing_y_data = [], []

for label in y_data:
    indexing_y_data.append(label2index[label])

In [10]:
tokenizer = Tokenizer(num_words=300) #300개 단어로 구성된 vocabulary 사용 

#x_data를 이용하여 vocabulary 생성
tokenizer.fit_on_texts(x_data)

#x_data의 각 문장들을 one-hot 벡터의 합으로 치환
indexing_x_data = tokenizer.texts_to_matrix(x_data, mode='count').tolist()

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# 200개 자질을 이용해 문장을 vectorize 
features = list(dict(features).keys())
feature_x_data = CountVectorizer(vocabulary=features).transform(x_data).toarray().tolist()

# 카이제곱 통계량으로 구한 자질을 indexing_x_data에 더하기
for i in range(len(indexing_x_data)):
    indexing_x_data[i] = indexing_x_data[i] + feature_x_data[i]

In [12]:
len(indexing_x_data[0])

500

## 5. SVM 분류기 학습시키기 

In [13]:
from sklearn.svm import SVC

# 전체 데이터를 9:1의 비율로 나누어 학습 및 평가 데이터로 사용
num_of_train_data = int(len(indexing_x_data)*0.9)

train_x, train_y = indexing_x_data[:num_of_train_data], indexing_y_data[:num_of_train_data]
test_x, test_y = indexing_x_data[num_of_train_data:], indexing_y_data[num_of_train_data:]

print("train_x의 개수 : " + str(len(train_x)))
print("train_y의 개수 : " + str(len(train_y)))
print("test_x의 개수 : " + str(len(test_x)))
print("test_y의 개수 : " + str(len(test_y)))

svm = SVC(kernel='rbf')
svm.fit(train_x, train_y)

train_x의 개수 : 1350
train_y의 개수 : 1350
test_x의 개수 : 150
test_y의 개수 : 150


SVC()

## 6. 성능 평가

In [14]:
from sklearn.metrics import accuracy_score

predict = svm.predict(test_x)

print('Accuracy: %.2f' % accuracy_score(test_y, predict))

Accuracy: 0.95
